# Quickstart

In this guide, we discuss how to deploy and interact with a Ray cluster on Kubernetes.

This guide is available at the Ray GitHub repo as an [executable Jupyter notebook](https://github.com/ray-project/ray/blob/master/doc/source/cluster/kuberay/quickstart.ipynb).


## Preparation

### Install Ray's latest release

In [ ]:
pip install -U ray[default]

See the [installation instructions](https://docs.ray.io/en/latest/ray-overview/installation.html) for more details. 

### Install Kubectl

We will use kubectl to interact with Kubernetes. Find installation instructions at the [Kubernetes documentation](https://kubernetes.io/docs/tasks/tools/#kubectl).

### Access a Kubernetes cluster

We will need access to a Kubernetes cluster. There are two options here:
1. Configure access to a remote Kubernetes cluster
**OR**
2. Run the examples locally by [installing kind](https://kind.sigs.k8s.io/docs/user/quick-start/#installation). Start your [kind](https://kind.sigs.k8s.io/) cluster by running

In [ ]:
! kind create cluster

To run the example in this guide, make sure your Kubernetes cluster can accomodate
additional resource requests of 4 CPU and 2.5Gi memory. 

## Deploying the KubeRay operator

Deploy the KubeRay Operator by cloning the KubeRay repo and applying the relevant configuration files from the master branch.  

In [ ]:
%%bash
# After the KubeRay 0.3.0 release branch cut, this documentation will be updated to refer to 0.3.0 branch.
git clone https://github.com/ray-project/kuberay
# This creates the KubeRay operator and all of the resources it needs.
kubectl create -k ray-operator/config/default
# Note that we must use "kubectl create" in the above command."kubectl apply" will not work due to https://github.com/ray-project/kuberay/issues/271

Confirm that the operator is running in the namespace `ray-system`.

In [ ]:
! kubectl -n ray-system get pod --selector=app.kubernetes.io/component=kuberay-operator
# NAME                                READY   STATUS    RESTARTS   AGE
# kuberay-operator-557c6c8bcd-t9zkz   1/1     Running   0          XXs

### Namespace-scoped operator
Note that the above command deploys the operator at _Kubernetes cluster scope_; the operator will manage resources in all Kubernetes namespaces.
**If your use-case requires running the operator at namespaced-scope**, refer to [the instructions at the KubeRay docs](https://github.com/ray-project/kuberay#single-namespace-version).

## Deploying a Ray Cluster

Once the KubeRay operator is running, we are ready to deploy a Ray cluster by deploying a RayCluster Custom Resource:

In [ ]:
# This Ray cluster is named `raycluster-autoscaler` because it has the optional Ray Autoscaler support enabled.
kubectl apply -f kuberay/ray-operator/samples/ray-cluster.autoscaler.yaml

Once the RayCluster CR has been created, you can view it by running

In [ ]:
! kubectl get raycluster
# NAME                    AGE
# raycluster-autoscaler   XXs

In [ ]:
! kubectl get pods --selector=ray.io/cluster=raycluster-autoscaler
# NAME                                             READY   STATUS    RESTARTS   AGE
# raycluster-autoscaler-head-xxxxx                 2/2     Running   0          XXs
# raycluster-autoscaler-worker-small-group-yyyyy   1/1     Running   0          XXs

We see a Ray head with two (2/2) containers -- the Ray container and autoscaler sidecar. We also have a Ray worker with its single Ray container.

## Interacting with a Ray Cluster

Now, let's interact with the Ray cluster we've deployed.

### Accessing the cluster with kubectl exec

The most straightforward way to experiment with your Ray cluster is to
exec directly into the head pod. First, identify your cluster's head pod:

In [1]:
! kubectl get pods --selector=ray.io/cluster=raycluster-autoscaler --selector=ray.io/node-type=head\
 -o custom-columns=POD:metadata.name --no-headers
# raycluster-autoscaler-head-xxxxx

raycluster-autoscaler-head-plqwd


Now, we can run a Ray program on the Ray head pod. The Ray program in the next cell asks the autoscaler to scale the cluster to a total of 3 CPUs. The head and worker in our example cluster each have a capacity of 1 CPU, so the request should trigger upscaling of an additional worker pod.

(Note that in real-life scenarios, we will typically want to use much larger Ray pods. In fact, it is advantageous to size each Ray pod to take up an entire Kubernetes node. See {ref} for more details.)

In [8]:
# Substitute your output from the last cell in place of "raycluster-autoscaler-head-xxxxx"
! kubectl exec raycluster-autoscaler-head-wpcth -it -c ray-head -- python -c\
 "import ray; ray.init(); ray.autoscaler.sdk.request_resources(num_cpus=3)"

### Autoscaling

The last command should have triggered Ray pod upscaling. To confirm the new worker pod is up, let's query the RayCluster's pods again:

In [3]:
! kubectl get pod --selector=ray.io/cluster=raycluster-autoscaler
# NAME                                             READY   STATUS    RESTARTS   AGE
# raycluster-autoscaler-head-xxxxx                 2/2     Running   0          XXs
# raycluster-autoscaler-worker-small-group-yyyyy   1/1     Running   0          XXs
# raycluster-autoscaler-worker-small-group-zzzzz   1/1     Running   0          XXs 

NAME                                             READY   STATUS            RESTARTS   AGE
raycluster-autoscaler-head-plqwd                 2/2     Running           0          5m58s
raycluster-autoscaler-worker-small-group-7kvxw   1/1     Running           0          5m58s
raycluster-autoscaler-worker-small-group-dtbkh   0/1     PodInitializing   0          1s


To get a summary of your cluster's status, run `ray status` on your cluster's Ray head node.

In [5]:
! kubectl exec raycluster-autoscaler-head-xxxxx -it -c ray-head -- ray status
# ======== Autoscaler status: 2022-07-21 xxxxxxxxxx ========
# ....

======== Autoscaler status: 2022-07-21 21:22:25.355156 ========
Node status
---------------------------------------------------------------
Healthy:
 1 head-group
 2 small-group
Pending:
 (no pending nodes)
Recent failures:
 (no failures)

Resources
---------------------------------------------------------------
Usage:
 0.0/3.0 CPU
 0.00/1.931 GiB memory
 0.00/0.446 GiB object_store_memory

Demands:
 {'CPU': 1}: 3+ from request_resources()


Alternatively, to examine the full autoscaling logs, fetch the stdout of the Ray head pod's autoscaler sidecar:

In [6]:
# This command gets the last 20 lines of autoscaler logs.
! kubectl logs raycluster-autoscaler-head-xxxxx -c autoscaler | tail -n 20
# ======== Autoscaler status: 2022-07-21 xxxxxxxxxx ========
# ...

======== Autoscaler status: 2022-07-21 21:23:15.315731 ========
Node status
---------------------------------------------------------------
Healthy:
 1 head-group
 2 small-group
Pending:
 (no pending nodes)
Recent failures:
 (no failures)

Resources
---------------------------------------------------------------
Usage:
 0.0/3.0 CPU
 0.00/1.931 GiB memory
 0.00/0.446 GiB object_store_memory

Demands:
 {'CPU': 1}: 3+ from request_resources()


### The Ray head service

The KubeRay operator configures a [Kubernetes service](https://kubernetes.io/docs/concepts/services-networking/service/) targeting the Ray head pod. This service allows us to interact with Ray clusters without directly executing commands in the Ray container. To identify the Ray head service for our example cluster, run

In [ ]:
! kubectl get service raycluster-autoscaler-head-svc
# NAME                             TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)                       AGE
# raycluster-autoscaler-head-svc   ClusterIP   10.96.114.20   <none>        6379/TCP,8265/TCP,10001/TCP   XXs

### Using Ray Job submission

Ray provides a [Job Submission API](https://docs.ray.io/en/master/cluster/job-submission.html#ray-job-submission) which can be used to submit Ray workloads to a remote Ray cluster. The Ray Job Submission server listens on the Ray head's Dashboard port, 8265 by default. Let's access the dashboard port via port-forwarding. 

Note: The following port-forwarding command is blocking. If you are following along from a Jupyter notebook, the command must be executed in a separate shell outside of the notebook.

In [ ]:
%%bash
# Execute this in a separate shell.
kubectl port-forward service/raycluster-autoscaler-head-svc 8265:8265

Now that we have forwarded the dashboard port to a local port, we can submit a Ray program for execution on the head node.

In [11]:
# Install Ray locally to use the Ray Job CLI
# The following job's logs will show the Ray cluster's total resource capacity, including 3 CPUs.
! ray job submit --address http://localhost:8265 -- python -c "import ray; ray.init(); print(ray.cluster_resources())"

Job submission server address: http://localhost:8265

-------------------------------------------------------
Job 'raysubmit_kyET4V7HNZ4etBTu' submitted successfully
-------------------------------------------------------

Next steps
  Query the logs of the job:
    ray job logs raysubmit_kyET4V7HNZ4etBTu
  Query the status of the job:
    ray job status raysubmit_kyET4V7HNZ4etBTu
  Request the job to be stopped:
    ray job stop raysubmit_kyET4V7HNZ4etBTu

Tailing logs until the job exits (disable with --no-wait):
{'CPU': 3.0, 'node:10.244.0.22': 1.0, 'object_store_memory': 482709502.0, 'memory': 2073741824.0, 'node:10.244.0.20': 1.0, 'node:10.244.0.21': 1.0}

------------------------------------------
Job 'raysubmit_kyET4V7HNZ4etBTu' succeeded
------------------------------------------



### Viewing the Ray Dashboard

Assuming the port-forwarding process described above is still running, you may view the Ray Dashboard by visiting `localhost:8265`.

The dashboard port will not be used in the rest of this guide. You may stop the port-forwarding process if you wish.

### Accessing the cluster using Ray Client

[Ray Client](https://docs.ray.io/en/latest/cluster/ray-client.html) allows you to interact programatically with a remote Ray cluster using the core Ray APIs.
To try out Ray Client, first make sure your local Ray version and Python minor version match the versions used in your Ray cluster. The Ray cluster in our example is running Ray 2.0.0 and Python 3.7, so that's what we'll need locally. If you have a different local Python version and would like to avoid changing it, you can modify the images specified in the yaml file `ray-cluster.autoscaler.yaml`. For example, use `rayproject/ray:2.0.0-py38` for Python 3.8.

After confirming the Ray and Python versions match up, the next step is to port-forward the Ray Client server port (10001 by default).
If you are following along in a Jupyter notebook, execute the following command in a separate shell.

In [ ]:
%%bash
# Execute this in a separate shell.
kubectl port-forward service/raycluster-autoscaler-head-svc 10001:10001

Now that we have port-forwarding set up, we can connect to the Ray Client from a local Python shell as follows:

In [17]:
import ray
import platform

# Disconnect from Ray if already connected.
ray.shutdown()

ray.init("ray://localhost:10001")

# The network name of the local machine.
local_host_name = platform.node()

@ray.remote
def get_host_name():
    return platform.node()

# The task will be scheduled on the head node.
# Thus, this variable will hold the head pod's name.
remote_host_name = ray.get(get_host_name.remote())

print("The local host name is {}".format(local_host_name))
print("The Ray head pod's name is {}".format(remote_host_name))

2022-07-21 21:38:29,779	INFO client_builder.py:247 -- Passing the following kwargs to ray.init() on the server: logging_format


The local host name is Dmitris-MBP.local.meter
The Ray head pod's name is raycluster-autoscaler-head-wpcth


## Cleanup

### Deleting a Ray Cluster
To delete the Ray Cluster we deployed in this example, you can run either of the following commands.

In [ ]:
# Delete by reference to the RayCluster custom resource
! kubectl delete raycluster raycluster-autoscaler

OR

In [ ]:
# Delete by reference to the yaml file we used to define the RayCluster CR 
! kubectl delete -f kuberay/ray-operator/samples/ray-cluster.autoscaler.yaml

Confirm that the Ray Cluster's pods are gone by running

In [ ]:
! kubectl get pods

Note that it may take several seconds for the Ray pods to be fully terminated.

### Deleting the KubeRay operator
In typical operation, the KubeRay operator should be left as a long-running process that manages many Ray clusters.
If you would like to delete the operator and associated resources, run

In [ ]:
! kubectl delete -k kuberay/ray-operator/config/default

### Deleting a local kind cluster
Finally, if you'd like to delete your local kind cluster, run

! kind delete cluster